In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os

In [2]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        # Extracting x and y coordinates from filename
        coords = [int(coord) for coord in self.images[idx].split('_')[:2]]
        return image, torch.tensor(coords, dtype=torch.float32)


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create dataset
dataset = CustomDataset(root_dir='/content/xyz2/xyz/apex', transform=transform)


In [6]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# **Resnet18**

In [7]:
class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        self.resnet = torchvision.models.resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 2)  # Output layer for 2 coordinates

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x
model = CustomResNet()


In [9]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


Epoch [1/10], Loss: 2596.5216
Epoch [2/10], Loss: 357.5792
Epoch [3/10], Loss: 271.7478
Epoch [4/10], Loss: 193.4144
Epoch [5/10], Loss: 189.6620
Epoch [6/10], Loss: 172.2707
Epoch [7/10], Loss: 134.2464
Epoch [8/10], Loss: 103.7409
Epoch [9/10], Loss: 97.9420
Epoch [10/10], Loss: 72.0884


# **MobilenetV3**

In [15]:
from torchvision.models import mobilenet_v3_small
class CustomMobileNetV3(nn.Module):
    def __init__(self):
        super(CustomMobileNetV3, self).__init__()
        self.mobilenet = mobilenet_v3_small(pretrained=True)
        self.fc = nn.Linear(1000, 2)  # Output layer for 2 coordinates

    def forward(self, x):
        x = self.mobilenet(x)
        x = self.fc(x)
        return x

# Initialize model
model = CustomMobileNetV3()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 141MB/s]


In [16]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


Epoch [1/10], Loss: 3093.3508
Epoch [2/10], Loss: 450.2945
Epoch [3/10], Loss: 304.7262
Epoch [4/10], Loss: 210.4437
Epoch [5/10], Loss: 148.1491
Epoch [6/10], Loss: 144.2715
Epoch [7/10], Loss: 127.2752
Epoch [8/10], Loss: 130.9740
Epoch [9/10], Loss: 109.7592
Epoch [10/10], Loss: 99.2242


In [17]:
# Evaluation
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, targets in test_loader:
        outputs = model(images)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * images.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


Test Loss: 359.5024


In [18]:
# Saving the Model
torch.save(model.state_dict(), '/content/model.pth')

In [ ]:
import matplotlib.pyplot as plt
# Plot function
def plot_image_with_dots(image, coords):
    plt.imshow(image.permute(1, 2, 0))
    plt.scatter(coords[0], coords[1], c='red', s=25)  # Plot red dot at predicted coordinates
    plt.show()

# Evaluation and plotting
with torch.no_grad():
    for images, targets in test_loader:
        outputs = model(images)
        for i in range(len(images)):
            image = images[i]
            coords = outputs[i].numpy().astype(int)  # Predicted coordinates
            plot_image_with_dots(image, coords)

In [4]:
!unzip /content/xyz_processed.zip -d /content/xyz2/

Archive:  /content/xyz_processed.zip
   creating: /content/xyz2/xyz/apex/
  inflating: /content/xyz2/xyz/apex/0_113_d642555c-1110-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/0_114_d6d681d2-1110-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/0_116_365de92a-1110-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/0_123_1e93be16-1113-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/0_128_72ca9efc-1111-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_100_8ff6a988-1113-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_104_75a82b42-1113-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_106_7ceccc3a-1110-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_106_855503dc-1111-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_108_26de8de8-1114-11ef-b1e0-a46bb6069316.jpg  
  inflating: /content/xyz2/xyz/apex/10_108_66e1906a-1110-11ef-b1e0-a46b